# 🎯 Guia Prático - Segmentação Semântica
## Para Prova Prática de Deep Learning

Este notebook contém templates e exemplos prontos para usar em provas práticas de segmentação semântica de imagens.

### O que é Segmentação Semântica?
- **Classificação pixel a pixel**: Cada pixel da imagem é classificado em uma categoria
- **Exemplo**: Separar foreground (gato/cachorro) de background em uma imagem
- **Aplicações**: Remoção de fundo, segmentação médica, detecção de objetos, etc.


## 📚 1. IMPORTS ESSENCIAIS


In [ ]:
# Imports básicos
import numpy as np
import matplotlib.pyplot as plt
import os
import glob
import tarfile
import urllib.request
import random

# TensorFlow/Keras
from tensorflow import keras
from keras import layers
from keras.utils import load_img, img_to_array, array_to_img

# Para visualização
plt.style.use('default')


## 📥 2. DOWNLOAD E CARREGAMENTO DE DADOS


In [ ]:
# 🔥 Função para baixar e extrair Oxford-IIIT Pets Dataset
def download_pets_dataset(data_dir="data"):
    """
    Baixa e extrai o dataset Oxford-IIIT Pets para segmentação semântica
    """
    os.makedirs(data_dir, exist_ok=True)
    
    # URLs do dataset
    images_url = 'http://www.robots.ox.ac.uk/~vgg/data/pets/data/images.tar.gz'
    annotations_url = 'http://www.robots.ox.ac.uk/~vgg/data/pets/data/annotations.tar.gz'
    
    print("📥 Baixando images.tar.gz...")
    images_path = os.path.join(data_dir, 'images.tar.gz')
    urllib.request.urlretrieve(images_url, images_path)
    
    print("📥 Baixando annotations.tar.gz...")
    annotations_path = os.path.join(data_dir, 'annotations.tar.gz')
    urllib.request.urlretrieve(annotations_url, annotations_path)
    
    # Extrair arquivos
    print("📦 Extraindo images.tar.gz...")
    with tarfile.open(images_path, 'r:gz') as tar:
        tar.extractall(data_dir)
    
    print("📦 Extraindo annotations.tar.gz...")
    with tarfile.open(annotations_path, 'r:gz') as tar:
        tar.extractall(data_dir)
    
    print("✅ Download e extração concluídos!")
    return data_dir

# Executar: download_pets_dataset()


In [ ]:
# 🔥 Função para preparar caminhos de imagens e máscaras
def prepare_dataset_paths(input_dir, target_dir):
    """
    Prepara listas de caminhos de imagens e máscaras correspondentes
    
    Args:
        input_dir: Diretório com imagens originais
        target_dir: Diretório com máscaras de segmentação
    
    Returns:
        input_img_paths: Lista de caminhos das imagens
        target_img_paths: Lista de caminhos das máscaras
    """
    # Obter todas as imagens JPG
    input_img_paths = sorted(glob.glob(os.path.join(input_dir, "*.jpg")))
    
    # Criar caminhos correspondentes das máscaras
    target_img_paths = []
    for img_path in input_img_paths:
        filename = os.path.basename(img_path)
        name_without_ext = os.path.splitext(filename)[0]
        mask_path = os.path.join(target_dir, name_without_ext + ".png")
        target_img_paths.append(mask_path)
    
    print(f"✅ Encontradas {len(input_img_paths)} imagens")
    print(f"📁 Primeiras 3 imagens: {input_img_paths[:3]}")
    print(f"📁 Primeiras 3 máscaras: {target_img_paths[:3]}")
    
    return input_img_paths, target_img_paths


In [ ]:
# 🔥 Função para carregar e pré-processar dataset completo
def load_segmentation_dataset(input_img_paths, target_img_paths, img_size=(200, 200), 
                             val_split=0.15, shuffle_seed=1337):
    """
    Carrega dataset completo de segmentação semântica
    
    Args:
        input_img_paths: Lista de caminhos das imagens
        target_img_paths: Lista de caminhos das máscaras
        img_size: Tamanho para redimensionar (altura, largura)
        val_split: Proporção para validação (ex: 0.15 = 15%)
        shuffle_seed: Seed para embaralhamento
    
    Returns:
        train_input_imgs, train_targets, val_input_imgs, val_targets
    """
    # Embaralhar mantendo correspondência
    random.Random(shuffle_seed).shuffle(input_img_paths)
    random.Random(shuffle_seed).shuffle(target_img_paths)
    
    # Funções auxiliares
    def path_to_input_image(path):
        return img_to_array(load_img(path, target_size=img_size))
    
    def path_to_target(path):
        img = img_to_array(
            load_img(path, target_size=img_size, color_mode="grayscale")
        )
        img = img.astype("uint8") - 1  # Labels: 0, 1, 2 (foreground, background, contour)
        return img
    
    # Carregar todas as imagens
    num_imgs = len(input_img_paths)
    input_imgs = np.zeros((num_imgs,) + img_size + (3,), dtype="float32")
    targets = np.zeros((num_imgs,) + img_size + (1,), dtype="uint8")
    
    print(f"📊 Carregando {num_imgs} imagens...")
    for i in range(num_imgs):
        if (i + 1) % 1000 == 0:
            print(f"  Processadas {i + 1}/{num_imgs} imagens...")
        input_imgs[i] = path_to_input_image(input_img_paths[i])
        targets[i] = path_to_target(target_img_paths[i])
    
    # Dividir em treino e validação
    num_val_samples = int(num_imgs * val_split)
    train_input_imgs = input_imgs[:-num_val_samples]
    train_targets = targets[:-num_val_samples]
    val_input_imgs = input_imgs[-num_val_samples:]
    val_targets = targets[-num_val_samples:]
    
    print(f"✅ Dataset carregado:")
    print(f"   Treino: {len(train_input_imgs)} imagens")
    print(f"   Validação: {len(val_input_imgs)} imagens")
    
    return train_input_imgs, train_targets, val_input_imgs, val_targets


## 🏗️ 3. MODELOS DE SEGMENTAÇÃO SEMÂNTICA


### 🔍 Arquitetura U-Net Style

**Conceito Principal:**
- **Encoder (Downsampling)**: Reduz tamanho, aumenta filtros (captura features)
- **Decoder (Upsampling)**: Aumenta tamanho, reduz filtros (reconstrói máscara)

**Por que não usar MaxPooling?**
- MaxPooling perde informação de localização espacial
- Em segmentação, precisamos manter localização precisa
- **Solução**: Usar convoluções com `strides=2` (downsample) e `Conv2DTranspose` com `strides=2` (upsample)


In [ ]:
# 🔥 TEMPLATE 1: Modelo básico de segmentação semântica (U-Net style)
def create_segmentation_model(img_size, num_classes):
    """
    Cria modelo básico para segmentação semântica
    
    Args:
        img_size: Tupla (altura, largura) da imagem
        num_classes: Número de classes (ex: 3 para foreground/background/contour)
    
    Returns:
        Modelo Keras compilado
    """
    inputs = keras.Input(shape=img_size + (3,))
    
    # Rescalar imagens para [0, 1]
    x = layers.Rescaling(1./255)(inputs)
    
    # ENCODER (Downsampling) - Captura features
    x = layers.Conv2D(64, 3, strides=2, activation="relu", padding="same")(x)
    x = layers.Conv2D(64, 3, activation="relu", padding="same")(x)
    
    x = layers.Conv2D(128, 3, strides=2, activation="relu", padding="same")(x)
    x = layers.Conv2D(128, 3, activation="relu", padding="same")(x)
    
    x = layers.Conv2D(256, 3, strides=2, activation="relu", padding="same")(x)
    x = layers.Conv2D(256, 3, activation="relu", padding="same")(x)
    
    # DECODER (Upsampling) - Reconstrói máscara
    x = layers.Conv2DTranspose(256, 3, activation="relu", padding="same")(x)
    x = layers.Conv2DTranspose(256, 3, activation="relu", padding="same", strides=2)(x)
    
    x = layers.Conv2DTranspose(128, 3, activation="relu", padding="same")(x)
    x = layers.Conv2DTranspose(128, 3, activation="relu", padding="same", strides=2)(x)
    
    x = layers.Conv2DTranspose(64, 3, activation="relu", padding="same")(x)
    x = layers.Conv2DTranspose(64, 3, activation="relu", padding="same", strides=2)(x)
    
    # Camada final: classificação por pixel
    outputs = layers.Conv2D(num_classes, 3, activation="softmax", padding="same")(x)
    
    model = keras.Model(inputs=inputs, outputs=outputs)
    return model


In [ ]:
### 🔥 TEMPLATE 2: Modelo mais profundo (opcional)


In [ ]:
# Template de modelo mais profundo (para imagens maiores ou problemas mais complexos)
def create_deep_segmentation_model(img_size, num_classes):
    """Modelo mais profundo com mais camadas"""
    inputs = keras.Input(shape=img_size + (3,))
    x = layers.Rescaling(1./255)(inputs)
    
    # ENCODER mais profundo
    x = layers.Conv2D(32, 3, strides=2, activation="relu", padding="same")(x)
    x = layers.Conv2D(32, 3, activation="relu", padding="same")(x)
    
    x = layers.Conv2D(64, 3, strides=2, activation="relu", padding="same")(x)
    x = layers.Conv2D(64, 3, activation="relu", padding="same")(x)
    
    x = layers.Conv2D(128, 3, strides=2, activation="relu", padding="same")(x)
    x = layers.Conv2D(128, 3, activation="relu", padding="same")(x)
    
    x = layers.Conv2D(256, 3, strides=2, activation="relu", padding="same")(x)
    x = layers.Conv2D(256, 3, activation="relu", padding="same")(x)
    
    # DECODER
    x = layers.Conv2DTranspose(256, 3, activation="relu", padding="same")(x)
    x = layers.Conv2DTranspose(256, 3, activation="relu", padding="same", strides=2)(x)
    
    x = layers.Conv2DTranspose(128, 3, activation="relu", padding="same")(x)
    x = layers.Conv2DTranspose(128, 3, activation="relu", padding="same", strides=2)(x)
    
    x = layers.Conv2DTranspose(64, 3, activation="relu", padding="same")(x)
    x = layers.Conv2DTranspose(64, 3, activation="relu", padding="same", strides=2)(x)
    
    outputs = layers.Conv2D(num_classes, 3, activation="softmax", padding="same")(x)
    
    model = keras.Model(inputs=inputs, outputs=outputs)
    return model


## 🎨 4. VISUALIZAÇÃO DE MÁSCARAS E RESULTADOS


In [ ]:
# 🔥 Função para visualizar máscaras de segmentação
def display_target(target_array):
    """
    Visualiza uma máscara de segmentação (normalizada para visualização)
    
    Args:
        target_array: Array com valores 0, 1, 2 (foreground, background, contour)
    """
    # Converter: (0,1,2) -> (0, 127, 254) para visualização
    normalized_array = (target_array.astype("uint8") - 1) * 127
    if len(normalized_array.shape) == 3:
        normalized_array = normalized_array[:, :, 0]
    
    plt.axis("off")
    plt.imshow(normalized_array, cmap='gray')
    plt.title("Segmentation Mask")

# Exemplo de uso:
# display_target(targets[0])


In [ ]:
# 🔥 Função para visualizar predições do modelo
def display_mask(pred_mask):
    """
    Visualiza máscara predita pelo modelo
    
    Args:
        pred_mask: Array de predição com shape (H, W, num_classes) ou (H, W)
    """
    # Se for tensor com múltiplas classes, pegar argmax
    if len(pred_mask.shape) == 3:
        mask = np.argmax(pred_mask,教育与=-1)
    else:
        mask = pred_mask
    
    # Normalizar para visualização: (0,1,2) -> (0, 127, 254)
    mask = mask * 127
    
    plt.axis("off")
    plt.imshow(mask, cmap='gray')
    plt.title("Predicted Mask")


## 🚀 5. WORKFLOW COMPLETO DE TREINAMENTO


In [ ]:
# 🔥 Workflow completo de treinamento de segmentação semântica
def complete_segmentation_workflow(img_size=(200, 200), num_classes=3, epochs=50, batch_size=64):
    """
    Workflow completo para treinamento de modelo de segmentação semântica
    
    Passos:
    1. Baixar dataset (se necessário)
    2. Preparar caminhos
    3. Carregar dados
    4. Criar modelo
    5. Compilar
    6. Treinar
    7. Avaliar e visualizar
    """
    
    # 1. Preparar caminhos (assumindo que dataset já foi baixado)
    input_dir = "data/images"
    target_dir = "data/annotations/trimaps"
    input_img_paths, target_img_paths = prepare_dataset_paths(input_dir, target_dir)
    
    # 2. Carregar dataset
    train_input_imgs, train_targets, val_input_imgs, val_targets = load_segmentation_dataset(
        input_img_paths, target_img_paths, img_size=img_size
    )
    
    # 3. Criar modelo
    model = create_segmentation_model(img_size, num_classes)
    model.summary()
    
    # 4. Compilar
    model.compile(optimizer="rmsprop", loss="sparse_categorical_crossentropy")
    
    # 5. Callbacks
    callbacks = [
        keras.callbacks.ModelCheckpoint(
            "segmentation_model.keras",
            save_best_only=True,
            monitor="val_loss"
        ),
        keras.callbacks.EarlyStopping(
            monitor="val_loss",
            patience=10,
            restore_best_weights=True
        )
    ]
    
    # 6. Treinar
    print("🚀 Iniciando treinamento...")
    history = model.fit(
        train_input_imgs, train_targets,
        epochs=epochs,
        batch_size=batch_size,
        validation_data=(val_input_imgs, val_targets),
        callbacks=callbacks
    )
    
    # 7. Visualizar resultados de algumas imagens
    print("\n📊 Visualizando resultados...")
    model_best = keras.models.load_model("segmentation_model.keras")
    
    # Visualizar algumas predições
    for i in range(min(3, len(val_input_imgs))):
        test_image = val_input_imgs[i]
        true_mask = val_targets[i]
        
        # Fazer predição
        pred_mask = model_best.predict(np.expand_dims(test_image, axis=0))[0]
        
        # Visualizar
        compare_segmentation(test_image, true_mask, pred_mask, title_prefix=f"Exemplo {i+1}: ")
    
    # 8. Plotar histórico
    epochs_range = range(1, len(history.history["loss"]) + 1)
    loss = history.history["loss"]
    val_loss = history.history["val_loss"]
    
    plt.figure(figsize=(10, 5))
    plt.plot(epochs_range, loss, "bo", label="Training loss")
    plt.plot(epochs_range, val_loss, "b", label="Validation loss")
    plt.title("Training and Validation Loss")
    plt.xlabel("Epochs")
    plt.ylabel("Loss")
    plt.legend()
    plt.grid(True)
    plt.show()
    
    return model_best, history

# Executar: model, history = complete_segmentation_workflow()


## 📋 6. DICAS RÁPIDAS PARA PROVA


### ⚡ Comandos Essenciais

```python
# 1. Verificar shapes dos dados
print("Input shape:", train_input_imgs.shape)  # (N, H, W, 3)
print("Target shape:", train_targets.shape)    # (N, H, W, 1)

# 2. Verificar valores das máscaras
print("Valores únicos na máscara:", np.unique(train_targets))  # Deve ser [0, 1, 2]

# 3. Fazer predição em uma imagem
pred = model.predict(np.expand_dims(test_image, axis=0))[0]
predicted_mask = np.argmax(pred, axis=-1)

# 4. Verificar shape da saída
print("Output shape:", model.output_shape)  # Deve ser (None, H, W, num_classes)
```

### 🎯 Checklist para Prova

- [ ] Dataset carregado corretamente (imagens e máscaras)
- [ ] Máscaras têm valores corretos (0, 1, 2 para 3 classes)
- [ ] Modelo criado com Encoder-Decoder (Conv2D + Conv2DTranspose)
- [ ] Última camada usa `Conv2D(num_classes, activation="softmax")`
- [ ] Loss: `sparse_categorical_crossentropy` (para targets inteiros)
- [ ] Modelo compilado e treinado
- [ ] Resultados visualizados corretamente

### 🔧 Troubleshooting Comum

**Erro: Shape mismatch**
- Verificar se input_shape do modelo corresponde ao tamanho das imagens
- Verificar se num_classes corresponde ao número de classes nas máscaras

**Erro: Loss não diminui**
- Verificar se máscaras estão normalizadas corretamente (0, 1, 2)
- Tentar learning rate menor
- Verificar se dados estão sendo carregados corretamente

**Predições muito ruins**
- Verificar se modelo tem camadas suficientes
- Aumentar número de épocas
- Verificar overfitting (val_loss > train_loss)

### 💡 Diferenças Importantes: Classificação vs Segmentação

| Classificação | Segmentação |
|--------------|-------------|
| Output: (batch, num_classes) | Output: (batch, H, W, num_classes) |
| Usa MaxPooling | Usa Conv2D com strides=2 |
| Flatten + Dense | Conv2DTranspose para upsampling |
| Categorical Crossentropy | Sparse Categorical Crossentropy |



In [ ]:
# 🔥 Função completa para comparar imagem original, máscara verdadeira e predição
def compare_segmentation(image, true_mask, pred_mask=None, title_prefix=""):
    """
    Visualiza imagem original, máscara verdadeira e predição lado a lado
    
    Args:
        image: Imagem original (array)
        true_mask: Máscara verdadeira (ground truth)
        pred_mask: Máscara predita (opcional)
    """
    num_cols = 3 if pred_mask is not None else 2
    
    plt.figure(figsize=(15, 5))
    
    # Imagem original
    plt.subplot(1, num_cols, 1)
    plt.axis("off")
    if len(image.shape) == 3 and image.shape[2] == 3:
        plt.imshow(image.astype("uint8"))
    else:
        plt.imshow(array_to_img(image))
    plt.title(f"{title_prefix}Original Image")
    
    # Máscara verdadeira
    plt.subplot(1, num_cols, 2)
    display_target(true_mask)
    plt.title(f"{title_prefix}True Mask")
    
    # Predição (se fornecida)
    if pred_mask is not None:
        plt.subplot(1, num_cols, 3)
        display_mask(pred_mask)
        plt.title procurou"{title_prefix}Predicted Mask")
    
    plt.tight_layout()
    plt.show()

# Exemplo de uso:
# compare_segmentation(val_input_imgs[0], val_targets[0], pred_mask)
